# Marco del problema

El objetivo es predecir el precio del airbnb utilizando los datos disponibles.

En este caso, estamos hablando de un problema supervisado, ya que los datos están etiquetados para el entrenamiento. Al ser un problema de predicción, estamos ante un caso de regresión, concretamente, de regresión múltiple, ya que tenemos varias características para predecir el valor. Como solamente tenemos que predecir un valor, estamos hablando de una regresión univariante.

Al tratarse de un problema de regresión, tomaremos como medida de confianza el error cuadrático medio **RMSE**

# Carga de datos

Cargamos los datos del dataset usando Pandas, creamos una función para ello

In [4]:
import pandas as pd
import os

# Carga el dataset y devuelve un dataframe de Pandas
def load_airbnb_dataset(ruta):
    csv_path = os.path.join(ruta, "airbnb-listings.csv")
    return pd.read_csv(csv_path, sep=';')

In [5]:
housing = load_airbnb_dataset('datasets')
housing.head

<bound method NDFrame.head of              ID                            Listing Url       Scrape ID  \
0      11210388  https://www.airbnb.com/rooms/11210388  20170306202425   
1      17471131  https://www.airbnb.com/rooms/17471131  20170407214050   
2      17584891  https://www.airbnb.com/rooms/17584891  20170407214050   
3       5398030   https://www.airbnb.com/rooms/5398030  20170407214050   
4      18104606  https://www.airbnb.com/rooms/18104606  20170407214050   
...         ...                                    ...             ...   
14775   2098030   https://www.airbnb.com/rooms/2098030  20170502132028   
14776  17989442  https://www.airbnb.com/rooms/17989442  20170404145355   
14777  13278634  https://www.airbnb.com/rooms/13278634  20170404145355   
14778  10733941  https://www.airbnb.com/rooms/10733941  20160516142622   
14779   4637801   https://www.airbnb.com/rooms/4637801  20160104002432   

      Last Scraped                                         Name  \
0       2017-0

Al usar la instrucción **head** de Pandas, obtenemos una primera visión de los datos. 

Cada fila corresponde a un apartamento, la primera fila contiene los nombres de las columnas.

# Estructura de los datos

Con la instrucción **înfo** vemos una pequeña descripción de los datos.

In [6]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14780 entries, 0 to 14779
Data columns (total 89 columns):
ID                                14780 non-null int64
Listing Url                       14780 non-null object
Scrape ID                         14780 non-null int64
Last Scraped                      14780 non-null object
Name                              14779 non-null object
Summary                           14189 non-null object
Space                             10888 non-null object
Description                       14774 non-null object
Experiences Offered               14780 non-null object
Neighborhood Overview             9134 non-null object
Notes                             5644 non-null object
Transit                           9066 non-null object
Access                            8318 non-null object
Interaction                       8228 non-null object
House Rules                       9619 non-null object
Thumbnail Url                     11960 non-null object
Med

Vemos que tenemos un total de 89 variables y 14780 observaciones (muy pocas para los estándares de Machine Learning).

Hay variables, por ejemplo **Has Availability** que solo tiene 12 valores no nulos.

La variable a predecir **price** (de ahora en adelante **y**) tiene 14763 valores no nulos, por lo que hay 17 pisos sin precio.

Hay muchas variables de tipo **object**, al leer desde un CSV, es posible que san variables de texto.

Cuando hicimos el head, pudimos ver que la variable "Cancellation Policy" tiene valores repetitivos, por lo que podría ser categórica. Vamos a comprobar si hay más variables categóricas.

In [9]:
# Hacer array para mostrar el value counts de cada una de las variable
print(housing[1][1])

KeyError: 1

In [7]:
housing.describe()

,ID,Scrape ID,Host ID,Host Response Rate,Host Listings Count,Host Total Listings Count,Latitude,Longitude,Accommodates,Bathrooms,...,Number of Reviews,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,Calculated host listings count,Reviews per Month
count,1.478000e+04,1.478000e+04,1.478000e+04,12881.000000,14777.000000,14777.000000,14780.000000,14780.000000,14780.000000,14725.000000,...,14780.000000,11476.000000,11454.000000,11460.000000,11443.000000,11460.000000,11440.000000,11439.000000,14776.000000,11618.000000
mean,1.028089e+07,2.017037e+13,3.608080e+07,94.823461,12.513636,12.513636,40.497626,-3.858041,3.277808,1.281732,...,22.632273,91.697978,9.416012,9.328883,9.621778,9.647033,9.532168,9.218201,9.486871,1.870014
std,5.564829e+06,5.667971e+08,3.425360e+07,15.215988,34.090223,34.090223,4.641387,14.123146,2.097291,0.658517,...,38.290244,8.989101,0.921938,0.989933,0.802736,0.767116,0.774527,0.950578,23.626014,1.867550
min,1.862800e+04,2.016010e+13,1.745300e+04,0.000000,0.000000,0.000000,-37.851182,-123.131344,1.000000,0.000000,...,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.020000
25%,5.554732e+06,2.017041e+13,6.787360e+06,100.000000,1.000000,1.000000,40.409726,-3.707604,2.000000,1.000000,...,1.000000,89.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,1.000000,0.450000
50%,1.133492e+07,2.017041e+13,2.464875e+07,100.000000,2.000000,2.000000,40.419466,-3.700785,3.000000,1.000000,...,7.000000,94.000000,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,2.000000,1.200000
75%,1.532631e+07,2.017041e+13,5.432919e+07,100.000000,6.000000,6.000000,40.430916,-3.684057,4.000000,1.000000,...,27.000000,98.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,5.000000,2.780000
max,1.910969e+07,2.017062e+13,1.247534e+08,100.000000,519.000000,519.000000,55.966912,153.371427,16.000000,8.000000,...,446.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,145.000000,17.210000
